# Pemanggilan Library yang dibutuhkan

In [9]:
from google.colab import drive
drive.mount('./MyDrive')

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Drive already mounted at ./MyDrive; to attempt to forcibly remount, call drive.mount("./MyDrive", force_remount=True).


# EDA

In [54]:
# Membaca dataset
df = pd.read_csv('/content/MyDrive/MyDrive/songs.csv')
df

,Name,Artist,Album,Popularity,Lyrics
0,Imagine - Remastered 2010,John Lennon,Imagine,79,8 ContributorsDiscographie 2021 — Pop & Rock L...
1,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,0,38 ContributorsA Whiter Shade of Pale Lyrics[I...
2,My Sweet Lord,George Harrison,All Things Must Pass (Remastered),0,58 ContributorsMy Sweet Lord Lyrics[Chorus]\nM...
3,God Only Knows - Mono,The Beach Boys,Pet Sounds (Original Mono & Stereo Mix),67,1 ContributorGod Only Knows (mono mix) LyricsI...
4,Bridge Over Troubled Water,Simon & Garfunkel,Bridge Over Troubled Water,72,57 ContributorsBridge Over Troubled Water Lyri...
...,...,...,...,...,...
794,Thinking out Loud,Ed Sheeran,x (Deluxe Edition),85,192 ContributorsTranslationsTürkçeEspañolPortu...
795,Sorry,Justin Bieber,Purpose (Deluxe),83,194 ContributorsTranslationsEspañolPortuguêsFr...
796,Lean On (feat. MØ & DJ Snake),Major Lazer,Major Lazer Essentials,30,3 ContributorsTop Tracks of 2016 - Global Lyri...
797,Take Me To Church,Hozier,The Singer Songwriter,25,339 ContributorsTranslationsفارسیEspañolFrança...


# Feature Engineering

In [60]:
# Convert 'Popularity' into discrete bins
df['PopularityBin'] = pd.cut(df['Popularity'], bins=[0, 20, 40, 60, 80, 100], labels=['Very Low', 'Low', 'Moderate', 'High', 'Very High'])
df

,Name,Artist,Album,Popularity,Lyrics,PopularityBin
0,Imagine - Remastered 2010,John Lennon,Imagine,79,8 ContributorsDiscographie 2021 — Pop & Rock L...,High
1,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,0,38 ContributorsA Whiter Shade of Pale Lyrics[I...,NaN
2,My Sweet Lord,George Harrison,All Things Must Pass (Remastered),0,58 ContributorsMy Sweet Lord Lyrics[Chorus]\nM...,NaN
3,God Only Knows - Mono,The Beach Boys,Pet Sounds (Original Mono & Stereo Mix),67,1 ContributorGod Only Knows (mono mix) LyricsI...,High
4,Bridge Over Troubled Water,Simon & Garfunkel,Bridge Over Troubled Water,72,57 ContributorsBridge Over Troubled Water Lyri...,High
...,...,...,...,...,...,...
794,Thinking out Loud,Ed Sheeran,x (Deluxe Edition),85,192 ContributorsTranslationsTürkçeEspañolPortu...,Very High
795,Sorry,Justin Bieber,Purpose (Deluxe),83,194 ContributorsTranslationsEspañolPortuguêsFr...,Very High
796,Lean On (feat. MØ & DJ Snake),Major Lazer,Major Lazer Essentials,30,3 ContributorsTop Tracks of 2016 - Global Lyri...,Low
797,Take Me To Church,Hozier,The Singer Songwriter,25,339 ContributorsTranslationsفارسیEspañolFrança...,Low


# Preprocessing & Make a Recommendation

In [61]:
# Membuat fungsi untuk rekomendasi lagu
def recommend_song(song_name, user_data, df, limit=5):
    # Memeriksa apakah lagu yang dicari ada dalam data
    if song_name not in df['Name'].values:
        return f"Lagu '{song_name}' tidak ditemukan dalam data."

    # Filter data untuk mendapatkan lagu dengan popularity lebih besar dari 0
    df = df[df['Popularity'] > 0]

    # Pivot table untuk mendapatkan matriks user-item
    user_item_matrix = df.pivot_table(index='Name', columns='PopularityBin', values='Popularity', fill_value=0)

    # Menghitung kesamaan korelasi Pearson
    similarity_matrix = cosine_similarity(user_item_matrix)

    # Mengambil popularity lagu yang diinginkan
    target_song_popularity = user_data['PopularityBin'].values[0]

    # Mendapatkan kesamaan korelasi untuk lagu yang diinginkan
    song_similarity = similarity_matrix[user_item_matrix.columns.get_loc(target_song_popularity)]

    # Mendapatkan lagu yang paling mirip
    similar_songs = df[df['PopularityBin'] == user_item_matrix.columns[song_similarity.argmax()]]

    # Menghapus lagu yang sedang dicari dari rekomendasi
    similar_songs = similar_songs[similar_songs['Name'] != song_name]

    # Batasi jumlah rekomendasi
    recommendations = similar_songs[['Name', 'Artist', 'Album', 'Popularity']].head(limit)

    return recommendations

# Testing Recommendation

In [64]:
song_to_recommend = 'Take Me To Church' # Lagu yang ingin dicari rekomendasinya
user_data = df[df['Name'] == song_to_recommend]
recommendations = recommend_song(song_to_recommend, user_data, df, limit=5) # Atur limit rekomendasi
print(recommendations)

                               Name                         Artist  \
28   Waterloo Sunset - Mono Version                      The Kinks   
68                       Walk on By                 Dionne Warwick   
428                     Reflections  The Band That Saved The World   
433           Queen of Belle la Vie  The Band That Saved The World   
438                      Old School  The Band That Saved The World   

                                                 Album  Popularity  
28                    Classics (The Best of The Kinks)          40  
68   The Dionne Warwick Collection: Her All-Time Gr...          39  
428                                     Name in Lights          33  
433                                     Name in Lights          33  
438                                     Name in Lights          33  
